# Meta Motivo benchmarking using HumEnv

This notebook shows how to evaluate a Meta Motivo model using the benchmark proposed in HumEnv. It assumes that motions have been processed and are available in the folder `MOTIONS_BASE_PATH`.

In [ ]:
from metamotivo.fb_cpr.huggingface import FBcprModel
from metamotivo.wrappers.humenvbench import RewardWrapper, TrackingWrapper, GoalWrapper
from metamotivo.buffers.buffers import DictBuffer
from huggingface_hub import hf_hub_download
import h5py
from humenv import ALL_TASKS
from humenv.bench import (
    RewardEvaluation,
    GoalEvaluation,
    TrackingEvaluation,
)


MOTIONS_BASE_PATH = "humenv/data_preparation/humenv_amass"
MOTIONS_INIT_STATE = "CMU_small1_train_0.1.txt"
MOTIONS_TRACKING = "large1_small1_test_0.1.txt"
GOAL_DICT = {}

Load inference buffer.

In [ ]:
buffer_path = hf_hub_download(
        repo_id="facebook/metamotivo-S-1",
        filename="data/buffer_inference_500000.hdf5",
        repo_type="model",
        local_dir="metamotivo-S-1-datasets",
    )
hf = h5py.File(buffer_path, "r")
data = {k: v[:] for k, v in hf.items()}
buffer = DictBuffer(capacity=data["qpos"].shape[0], device="cpu")
buffer.extend(data)

Load model and prepare it for inference.

In [ ]:
model = FBcprModel.from_pretrained("facebook/metamotivo-S-1")
model = RewardWrapper(
        model=model,
        inference_dataset=buffer,
        num_samples_per_inference=10_000,
        inference_function="reward_wr_inference",
        max_workers=10,
    )
model = GoalWrapper(model=model)
model = TrackingWrapper(model=model)

Humenv provides 3 evaluation protocols:
- reward based,
- goal based,
- tracking

In [ ]:
reward_eval = RewardEvaluation(
        tasks=ALL_TASKS,
        env_kwargs={
            "state_init": "MoCapAndFall",
        },
        num_contexts=1,
        num_envs=10,
        num_episodes=100,
        motion_base_path=MOTIONS_BASE_PATH,
        motions=MOTIONS_INIT_STATE,
    )
goal_eval = GoalEvaluation(
    goals=GOAL_DICT,
    env_kwargs={
        "state_init": "MoCapAndFall",
    },
    num_contexts=1,
    num_envs=10,
    num_episodes=100,
    motion_base_path=MOTIONS_BASE_PATH,
    motions=MOTIONS_INIT_STATE,
)
tracking_eval = TrackingEvaluation(
    motions=MOTIONS_TRACKING,
    motion_base_path=MOTIONS_BASE_PATH,
    env_kwargs={
        "state_init": "Default",
    },
    num_envs=10,
)

In [ ]:
reward_metrics = reward_eval.run(agent=model)
print(reward_metrics)

In [ ]:
goal_metrics = goal_eval.run(agent=model)
print(goal_metrics)

In [ ]:
tracking_metrics = tracking_eval.run(agent=model)
print(tracking_metrics)